In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy.sparse as sparse

from lightfm import LightFM
import lightgbm as lgb
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [18]:
train = pd.read_csv('data/train.csv.zip', low_memory=False)
test = pd.read_csv('data/test.csv.zip', low_memory=False)
sample_submission = pd.read_csv('data/sample_submission.csv.zip', low_memory=False)

In [19]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [20]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [21]:
model = LightFM(learning_rate=0.07, 
                loss='logistic',
                no_components=40, 
                learning_schedule='adagrad',
               random_state = 13)
model = model.fit(ratings_coo, epochs=23, 
                  num_threads=8)

preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)
sklearn.metrics.roc_auc_score(test_data.rating, preds)

0.7469984813758245

In [22]:
preds = model.predict(test.userid.values,
                      test.itemid.values)

In [23]:
preds.min(), preds.max()

(-20.87497329711914, 26.767932891845703)

In [24]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [25]:
normalized_preds.min(), normalized_preds.max()


(0.0, 1.0)

In [26]:
sample_submission['rating'] = normalized_preds
sample_submission.to_csv('submission_4.csv', index=False)
sample_submission.head(3)

,Id,rating
0,0,0.476275
1,1,0.468919
2,2,0.448274


In [27]:
train_data_GB = pd.DataFrame({
    'userid': train['userid'],
    'itemid': train['itemid'],
    'rating': train['rating']
})

test_data_GB = pd.DataFrame({
    'userid': test['userid'],
    'itemid': test['itemid'],
})

In [28]:
train_data_GB_train, test_data_GB_test = train_test_split(train_data_GB,random_state=13, shuffle=True)

In [29]:
y_train = train_data_GB_train['rating']
y_test = test_data_GB_test['rating']
X_train = train_data_GB_train.drop('rating', axis=1)
X_test = test_data_GB_test.drop('rating', axis=1)

In [30]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'l1'},
    'num_leaves': 8,
    'learning_rate': 0.4,
    'feature_fraction': 0.1,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', sklearn.metrics.roc_auc_score(y_test, y_pred))

y____ =  gbm.predict(test_data_GB, num_iteration=gbm.best_iteration)

Starting training...
[1]	valid_0's l2: 0.127653	valid_0's l1: 0.256156
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.127582	valid_0's l1: 0.256108
[3]	valid_0's l2: 0.127545	valid_0's l1: 0.256056
[4]	valid_0's l2: 0.12752	valid_0's l1: 0.256012
[5]	valid_0's l2: 0.127492	valid_0's l1: 0.255964
[6]	valid_0's l2: 0.127469	valid_0's l1: 0.255816
[7]	valid_0's l2: 0.127457	valid_0's l1: 0.255719
[8]	valid_0's l2: 0.127454	valid_0's l1: 0.255661
[9]	valid_0's l2: 0.127416	valid_0's l1: 0.25559
[10]	valid_0's l2: 0.127406	valid_0's l1: 0.255551
[11]	valid_0's l2: 0.127389	valid_0's l1: 0.255669
[12]	valid_0's l2: 0.127367	valid_0's l1: 0.255722
[13]	valid_0's l2: 0.12736	valid_0's l1: 0.255746
[14]	valid_0's l2: 0.127362	valid_0's l1: 0.255769
[15]	valid_0's l2: 0.127358	valid_0's l1: 0.255777
[16]	valid_0's l2: 0.127341	valid_0's l1: 0.255679
[17]	valid_0's l2: 0.127326	valid_0's l1: 0.255616
[18]	valid_0's l2: 0.127322	valid_0's l1: 0.255573
[19]	valid_0

In [31]:
c = (normalized_preds  + y____) / 2